In [2]:
import numpy as np
import time
import pickle
import random
import math

FEATURE_NUM = 123
def loadDataSet(filename):
        dataMat = []
        labelMat = []
        fr = open(filename, 'r')
        for line in fr.readlines():
            lineArr = line.strip().split(' ')
            vec = np.zeros(FEATURE_NUM)
            for v in lineArr[1:]:
                feature_no = int(v.split(':')[0])
                feature_value = int(v.split(':')[1])
                vec[feature_no-1] = feature_value
            dataMat.append(vec)
            labelMat.append(int(lineArr[0]))
        return np.array(dataMat), np.array(labelMat)
        
def predict(dataMatrix, classLabels, weights, bias):
    m, n = np.shape(dataMatrix)
    right = 0
    wrong = 0
    for i in range(m): 
        rst = np.sum(np.dot(weights,dataMatrix[i]))+ bias
        if rst >= 1:
            rst = 1
        else:
            rst = -1
        if rst == classLabels[i]:
            right += 1
        else:
            wrong += 1
    return right, wrong

def derivateLw(dataMatrix, classLabels, weights, bias, C):
    condition = 1 - np.dot(classLabels, np.matmul(dataMatrix, np.transpose(weights))+bias)
    compare = np.less(condition, 0)
    gw = np.zeros(FEATURE_NUM)
    for i, c in enumerate(compare):
        if not c:
            gw[i] -= (classLabels[i]*dataMatrix[i])
    Lw = weights + (C/len(dataMatrix))*gw
    return Lw

def derivateLb(dataMatrix, classLabels, weights, bias, C):
    condition = 1 - np.dot(classLabels, np.matmul(dataMatrix, np.transpose(weights))+bias)
    compare = np.less(condition, 0)
    gb = 0.0
    for i, c in enumerate(compare):
        if not c:
            gb[i] -= classLabels[i]
    Lb = (C/len(dataMatrix))*gb
    return Lb

def NAG(dataMatrix, classLabels, weights, bias, k, alpha, v):
    C = 64
    v = k*v - alpha * derivateLw(dataMatrix, classLabels, weights, C)
    weights = weights + v
    bias = bias - lr*derivateLb(dataMatrix, classLabels, weights, bias, C)
    return weights, bias, v

def stocGradDescent(dataMatrix, classLabels, weights, bias):
    m, n = np.shape(dataMatrix)
    lr = 0.01
    C = 64
    weights = weights - lr*derivateLw(dataMatrix, classLabels, weights, bias, C)
    bias = bias - lr*derivateLb(dataMatrix, classLabels, weights, bias, C)
    return weights, bias
    
def saveModel(object):
    saveDir = "F:\\实验\\lab2\\svm_model_"+str(time.strftime('%Y-%m-%d-%H-%M-%S',time.localtime(time.time())))+".m"
    with open(saveDir, 'wb') as f:
        pickle.dump(object, f)
        print (saveDir + " Save Success!")

def loadModel(filename):
    with open(filename, 'rb') as f:
        param_list = pickle.load(f)
        return param_list[0], param_list[1]

def train(preModelName=""):
    weights = np.random.randn(1,FEATURE_NUM)
    v = 0
    k = 0.3
    alpha = 0.7
    bias = np.random.randn()
    if preModelName == "":
        pass
    else:
        weights, bias = pickle.load(preModelName)
    dMat_train, lMat_train = loadDataSet("F:\\实验\\lab2\\a9a_train")                                                         
    batch_num = 128
    iter_num = 10000
    for i in range(iter_num):
        if i%20000 == 0 and i!=0:
            saveModel([weights, bias])
        stocSelect = random.sample(list(np.arange(len(dMat_train))),batch_num)
        miniBatch = dMat_train[stocSelect]
        miniLabels = lMat_train[stocSelect]
        weights, bias, v = NAG(miniBatch, miniLabels, weights, bias, k, alpha, v)
        # weights, bias = stocGradDescent(miniBatch, miniLabels, weights, bias)
        right,wrong = predict(miniBatch, miniLabels, weights, bias)
        print ("iter:"+str(i)+" train acc:"+str(right/(right+wrong)))
    dMat_test, lMat_test = loadDataSet("F:\\实验\\lab2\\a9a.t")
    right,wrong = predict(dMat_test, lMat_test, weights, bias)
    print ("test acc:"+str(right/(right+wrong)))
    
if __name__ == "__main__":
    train()

ValueError: setting an array element with a sequence.